In [1]:
from sources.bdi_components.plans import PlanLibrary
from sources.bdi_components.belief import BeliefBase, State
from sources.bdi_components.inference import NLIModel
from sources.agent.agent import BDIAgent

from sources.agent.scienceworld import parse_observation

from scienceworld import ScienceWorldEnv
import re
from sources.agent.scienceworld import parse_observation

In [2]:
hg_model_hub_name = "alisawuffles/roberta-large-wanli"
nli_model = NLIModel(hg_model_hub_name)

In [3]:
# MAIN GOAL
main_plan = """
    IF your goal is to boil water THEN
        move to kitchen,
        pick up thermometer,
        get metal pot,
        fill metal pot with water,
        heat water on stove,
        focus on substance in metal pot,
"""

# get metal pot
subplan_a = """
    IF your goal is to get metal pot CONSIDERING you are in the kitchen AND you see a cupboard THEN
        open cupboard,
        pick up metal pot,
"""

# fill metal pot with water
subplan_b = """
    IF your goal is to fill metal pot with water CONSIDERING you have metal pot in your inventory AND you see a sink THEN
        move metal pot to sink,
        activate sink,
        deactivate sink,
        pick up metal pot,
"""

# heat water on stove
subplan_c = """
    IF your goal is to heat water on stove CONSIDERING you have metal pot with water AND you see a stove THEN
        move metal pot to stove,
        activate stove,
        wait,
        use thermometer in inventory on substance in metal pot,
        use thermometer in inventory on substance in metal pot,
        focus on substance in metal pot,
        wait,
        use thermometer in inventory on substance in metal pot
"""

subplan_d = """
    IF your goal is to move to kitchen CONSIDERING you are in art studio THEN
        open door,
        go to hallway,
        open door to kitchen,
        go to kitchen
"""

all_plans = [main_plan, subplan_a, subplan_b, subplan_c, subplan_d]

pl = PlanLibrary()
pl.load_plans_from_strings(all_plans)
print(pl.plans.keys())

dict_keys(['boil water', 'get metal pot', 'fill metal pot with water', 'heat water on stove', 'move to kitchen'])


In [4]:
env = ScienceWorldEnv("", "", envStepLimit=100)
randVariationIdx = 0 # water
env.load('boil', randVariationIdx)
# Reset the environment
observation, info = env.reset()

print(f"Task Name: " + 'boil' + " variation " + str(randVariationIdx))
print("Task Description: " + str(env.getTaskDescription()))
randVariationIdx = 0 # tin
env.load('boil', randVariationIdx)
# Reset the environment
observation, info = env.reset()

Task Name: boil variation 0
Task Description: Your task is to boil water. For compounds without a boiling point, combusting the substance is also acceptable. First, focus on the substance. Then, take actions that will cause it to change its state of matter.


In [5]:
def parse(look: str) -> list[str]:
    x = re.search(r"([\S\s]*?)(?:In it, you see:)([\S\s]*?)(?:You also see:)([\S\s]*)", look)
    if x == None:
        x = re.search(r"([\S\s]*?)(?:Here you see:)([\S\s]*?)(?:You also see:)([\S\s]*)", look)
    groups = x.groups()

    location = groups[0]
    objects = groups[1]
    doors = groups[2]

    loc_split = [location.strip()]
    obs_split = [obs.strip() for obs in objects.split('\n') if len(obs.strip()) > 0]
    obs_split = [f"You see {obs}" for obs in obs_split]
    doors_split = [door.strip() for door in doors.split('\n') if len(door.strip()) > 0]
    return loc_split + obs_split + doors_split

observation, reward, isCompleted, info = env.step('look around')
#root_event = env.getTaskDescription().split('.')[0].lower()
root_event = 'boil water'
print(f"Root event: {root_event}")
parsed_obs = parse(info['look'])
parsed_obs

Root event: boil water


['This room is called the art studio.',
 'You see the agent',
 'You see a substance called air',
 'You see a large cupboard. The large cupboard door is closed.',
 'You see a table. On the table is: a bowl (containing nothing).',
 'You see a wood cup (containing blue paint)',
 'You see a wood cup (containing red paint)',
 'You see a wood cup (containing yellow paint)',
 'A door to the hallway (that is closed)']

In [6]:
inventory = info['inv'].replace('\n', ' ').replace('\t', '')
current_state = State(goal=root_event, observation=observation, look=parsed_obs, inventory=inventory)
current_state

State(goal='boil water', observation='This room is called the art studio. In it, you see: \n\tthe agent\n\ta substance called air\n\ta large cupboard. The large cupboard door is closed. \n\ta table. On the table is: a bowl (containing nothing).\n\ta wood cup (containing blue paint)\n\ta wood cup (containing red paint)\n\ta wood cup (containing yellow paint)\nYou also see:\n\tA door to the hallway (that is closed)', look=['This room is called the art studio.', 'You see the agent', 'You see a substance called air', 'You see a large cupboard. The large cupboard door is closed.', 'You see a table. On the table is: a bowl (containing nothing).', 'You see a wood cup (containing blue paint)', 'You see a wood cup (containing red paint)', 'You see a wood cup (containing yellow paint)', 'A door to the hallway (that is closed)'], inventory='In your inventory, you see: an orange ')

In [7]:
def step_function(action: str) -> State:
    observation, reward, isCompleted, info = env.step(action)
    parsed_obs = parse(info['look'])
    inventory = info['inv'].replace('\n', ' ').replace('\t', '')

    print(f"Action {action} executed -> obs perceived {observation}")
    return State(goal=root_event, observation=observation, look=parsed_obs, inventory=inventory)


agent = BDIAgent(plan_library=pl, nli_model=nli_model)
agent.act(current_state, info['valid'], step_function=step_function)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


-> Event move to kitchen
drill down to event: move to kitchen
-> Event open door
Executing action open door
-> Event go to hallway
event go to hallway not recognized as a subgoal neither an action
-> Event pick up thermometer
event pick up thermometer not recognized as a subgoal neither an action


In [12]:
[act for act in info['valid'] if act.startswith("open")]

['open cupboard', 'open door', 'open drawer']

In [13]:
for i, action in enumerate(plan):
    observation, reward, isCompleted, info = env.step(action)
    print(f"---- Step {i} | action: {action} | Finish {isCompleted} | reward {reward} -----")
    print(observation)

'This room is called the art studio. In it, you see: \n\tthe agent\n\ta substance called air\n\ta large cupboard. The large cupboard door is closed. \n\ta table. On the table is: a bowl (containing nothing).\n\ta wood cup (containing blue paint)\n\ta wood cup (containing red paint)\n\ta wood cup (containing yellow paint)\nYou also see:\n\tA door to the hallway (that is closed)\n'

In [18]:
pl.plans['move to kitchen'][0].body

['open door', 'go to hallway', 'open door to kitchen', 'go to kitchen']

In [8]:
pl.plans['get metal pot']

[Plan(triggering_event='get metal pot', context=['you are in the kitchen AND you see a cupboard'], body=['open cupboard', 'pick up metal pot', ''], idx=1)]